In [6]:
#!pip install streamlit
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import LinearRegression
import xlsxwriter
# import matplotlib.pyplot as plt

# Clustering
from sklearn.cluster import KMeans
# conda install -c conda-forge scikit-learn-extra
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import itemgetter

# Principal Components Analysis
# from scipy import stats
# from sklearn.decomposition import PCA

# Classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import itertools

## Import Data

In [7]:
# Import Data
df = pd.read_csv('dgn_raw_data.csv')

# Add very small random number to Rating
df['target']=df['Rating'].apply(lambda x: x+random.random()/1000)

## Regressions for Each UID

In [8]:
# Unique IDs
ids = df.UID.unique()

# Run linear regressions for each UID
op = pd.DataFrame
intercept = []
coefficients=[]
UID = []
for p in ids:
    df_i = df[df.UID == p]              # Create dataframe for current user id
    X = df_i.filter(regex='^[a-zA-Z][0-9]')  # df input variables only
    y = df_i['target']                  # Series of target variable
    reg = LinearRegression().fit(X, y)  # Fit linear regression
    reg.score(X, y)                     # Score regression model
    unique_id=df_i['UID'].unique()      # Saves current user id
    const = reg.intercept_              # Save intercept of the regression model
    coef = reg.coef_                    # Coefficients of regression model
    UID.append(unique_id)               # Append current user id
    intercept.append(const)             # Append current intercept
    coefficients.append(coef)           # Append current regression coefficients

# Convert newly created lists into dataframes
intercep_new = pd.DataFrame(intercept)
coefficients_new = pd.DataFrame(coefficients)
UID_new = pd.DataFrame(UID)

# Get columns names
colNames = df.drop(['Rating', 'target',], axis=1).columns
colNames = colNames.insert(1, 'Const')
colNames

# Concatenate the new dataframes and add column names
op = pd.concat([UID_new,intercep_new, coefficients_new], axis=1)
op.columns = colNames

# Save only regression coefficients for clustering
scores = op.drop(['UID','Const'], axis=1)

## Cluster on Regression Coefficients

In [20]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_multiple.xlsx', engine='xlsxwriter')

# Create dataframe for storing all cluster/variable combo averages and stdevs
cls_averages_all = pd.DataFrame()

# All maps for all cluster solutions
all_maps = []

# Column of last cluster solution
first_var = 18
last_var = op.shape[1]

# Maximum number of clusters
max_clusters = 6

for n in range(2, max_clusters+1):
    
    sw = []
    
    # Create clustering objects
    cls1 = KMeans(n_clusters=n, random_state=0)
    cls2 = KMedoids(n_clusters=n, random_state=0)
    cls3 = AgglomerativeClustering(n_clusters=n,
                                   affinity='euclidean',
                                   linkage='ward')
        # Agglomerative clustering: if linkage=ward, affinity must be Euclidean
    cls_algs = [['kMeans', cls1],
                ['kMedoids', cls2],
                ['Hierarchical', cls3]]
    
    # Fit and score clustering solutions for i clusters w/ each algorithm
    for cls in cls_algs:
        
        # Fit the model to the factor analysis scores
        cls[1].fit(scores)
        
        # List of assigned clusters
        clusters = cls[1].fit_predict(scores)
        
        # Silhouette scores for each solution
        silhouette_avg = silhouette_score(scores,clusters)
        
        # Store solution info
        algorithm = cls[0]
        n_stats = [algorithm, n, silhouette_avg, clusters]
        sw.append(n_stats)

    # Reorder cluster lists by descending silhouette scores.
    # Clusters in first element should be assigned to training data.
    sw = sorted(sw, key=itemgetter(2), reverse=True)
    op[f'Optimal {sw[0][1]} cluster solution ({sw[0][0]})'] = sw[0][3] + 1
    
    
    #**********************************************************************#
    # This is where the classification stuff begins

for i in range(18,last_var):


    df_cl = op.iloc[:,np.r_[2:18,i]]  # i is the current cluster solution
    df_cl_cons = op.iloc[:,np.r_[1:18,i]]  # Same as df_cl but with constant

    #**********************************************************************#

    # Split data into 70% training, 30% validation
    train, valid = train_test_split(df_cl, test_size=0.30, random_state=123)

    # X is unlabeled training data, y is true training labels 
    X, y = train.iloc[:,0:-1], train.iloc[:,-1]

    X_valid, y_valid = valid.iloc[:,0:-1], valid.iloc[:,-1]

    #**********************************************************************#

    # Get variable importances

    clf1 = RandomForestClassifier(random_state=0)
    clf2 = GradientBoostingClassifier(random_state=0)

    classifiers = [['rf', clf1], ['gbt', clf2]]

    for classifier in classifiers:    
        # Fit classifier to training data
        classifier[1].fit(X,y)    

    # Create variable importance dataframe
    num_vars = list(range(1,len(clf1.feature_importances_)+1))
    importance = pd.DataFrame({'variable': num_vars,
                               'rf': clf1.feature_importances_,
                               'gbt': clf2.feature_importances_,})

    # Average variable importance of rf and gbt models
    importance['avg'] = (importance['rf']+importance['gbt'])/2

    # Put avg importances on a scale from 0 to 1 to make it easier to visualize
    importance['Relative Importance'] = np.interp(importance['avg'],
                                                  (importance['avg'].min(),
                                                   importance['avg'].max()),
                                                  (0, 1))

    # View top 10 variables when RF and GBT models are averaged
    top_10_avg = importance.sort_values(by='avg', ascending=False)[['avg','Relative Importance']].head(10)

    # Add variable rank column to dataframe
    importance_rank = num_vars
    importance = importance.sort_values(by='Relative Importance', ascending=False)
    importance['rank'] = importance_rank
    importance.reset_index(inplace=True)

    # Save index of top 5 variables (not the variable number!)
    top_5 = importance[importance['rank'] <= 5]['index']

    #**********************************************************************#
    # Average and Standard Deviations for each cluster/variable combination
    # For cluster 1 of 2, calculate the average and stdev for each variable
    # For cluster 2 of 2, calculate the average and stdev for each variable
    # Etc.

    if n == max_clusters:

        cls_avg_list = []

        # Take the mean of every variable for each cluster
        for k in range(1, df_cl_cons.iloc[:,-1].max()+1):
            cls_mean = df_cl_cons[df_cl_cons.iloc[:,-1] == k].iloc[:,0:-1].mean()
            cls_mean = cls_mean.append(pd.Series({"Count":df_cl[df_cl.iloc[:,-1] == k].iloc[:,0:-1].shape[0]}))
            cls_avg_list.append(cls_mean)
            cls_std = df_cl_cons[df_cl_cons.iloc[:,-1] == k].iloc[:,0:-1].std()
            cls_avg_list.append(cls_std)
            # NaN means there is either only 1 observation in that cluster or none.
        
        # Convert to dataframe and transpose
        cls_averages = pd.DataFrame(cls_avg_list)
        cls_averages = cls_averages.T

        # Create helpful column names (Cluster # of total_#)
        col_names = []
        
        for col in range(1, k+1):
            new_name1 = f"Avg cluster {col}/{k}"
            col_names.append(new_name1)
            new_name2 = f"Std cluster {col}/{k}"
            col_names.append(new_name2)            
            
        # Rename columns
        cls_averages.columns = col_names

        cls_averages_all = pd.concat([cls_averages_all, cls_averages], axis=1)
        
        


    #**********************************************************************#
    # Convert data to binary, train classifiers, score validation, create maps

    # Convert X, X_valid, and df_cl predictors to all 1 and -1
    X = (X.mask(df > 0, other=1, inplace=False)
         .mask(df <= 0, other=-1, inplace=False))
    X_valid = (X_valid.mask(df > 0, other=1, inplace=False)
               .mask(df <= 0, other=-1, inplace=False))
    all_data_masked = (df_cl.iloc[:,0:-1].mask(df > 0, other=1, inplace=False)
                       .mask(df <= 0, other=-1, inplace=False))

    map_collection = []

    # Retrain on the 2-5 most important variables
    for j in range(2,6):

        clf_scores = []

        clf1 = RandomForestClassifier(random_state=0)
        clf2 = GradientBoostingClassifier(random_state=0)
        clf3 = SVC(random_state=0)
        clf4 = KNeighborsClassifier()

        classifiers = [['rf', clf1], ['gbt', clf2], ['svc', clf3], ['knn', clf4]]

        # Fit each classifier to the current variable/cluster combination
        for classifier in classifiers:

            # Fit classifier to training data
            classifier[1].fit(X.iloc[:,np.r_[top_5[0:j]]],y)

            # Store classifier-specific results [algorithm object, classifier name, scores]
            results = [classifier[1],
                       classifier[0],
                       classifier[1].score(X_valid.iloc[:,np.r_[top_5[0:j]]],y_valid)]

            # Overall classifier results
            clf_scores.append(results)

        # Sort classifier accuracy in descending order
        clf_scores = sorted(clf_scores, key=itemgetter(2), reverse=True)
        # clf_scores[0][0] is the best model

        # Fit the best model on all data
        best_model = clf_scores[0][0].fit(all_data_masked.iloc[:,np.r_[top_5[0:j]]], df_cl.iloc[:,-1])

        #******************************************************************#
        # Create mappings

        # Creates grid of dimension j
        grid = pd.DataFrame(list(itertools.product([-1,1], repeat=j)))

        grid.columns = all_data_masked.iloc[:,np.r_[top_5[0:j]]].columns

        # This is the best model predicting the grid
        preds = best_model.predict(grid)            

        # Add to grid dataframe
        grid['Predicted Cluster'] = preds

        # Change grid to mapping to fit into the rest of the code
        mapping = grid

        # Save current mapping to map collection for this cluster solution
        map_collection.append(mapping)

        # Write each dataframe to a different worksheet.
        mapping.to_excel(writer, index=False, sheet_name=f"{df_cl.columns[-1][8:17]}s, {j} vars, {round(clf_scores[0][2]*100)}% Acc.")

    all_maps.append(map_collection)

op.to_excel(writer, index=False, sheet_name="All Regressions, Clusters")


# Add averages for all observations to cls_averages_all before exporting
all_obs = []

# Variable means for all observations
all_obs_mean = list(op.filter(regex='^[a-zA-Z][0-9]').mean().values)
all_obs_mean.insert(0,op['Const'].mean())
all_obs.append(all_obs_mean)

cls_averages_all['new_col'] = pd.Series(list(op.filter(regex='^[a-zA-Z][0-9]').mean().values))

# Variable standard deviations for all observations
all_obs_std = list(op.filter(regex='^[a-zA-Z][0-9]').std().values)
all_obs_std.insert(0,op['Const'].std())
all_obs.append(all_obs_std)


# Save as dataframe and append to all cls_averages_all dataframe
all_obs_cols = list(op.filter(regex='^[a-zA-Z][0-9]').columns)
all_obs_cols.insert(0, "Const")
all_obs_df = pd.DataFrame(all_obs, columns=all_obs_cols)
all_obs_df = all_obs_df.T
all_obs_cols = ['All obs avg', 'All obs stdev']
all_obs_df.columns = all_obs_cols
cls_averages_all = pd.concat([cls_averages_all, all_obs_df], axis=1)


cls_averages_all.to_excel(writer, sheet_name="Cluster Avgs and StDevs")

writer.save()

## End of script. Code below is for experimentation.

In [ ]:
# This is for making the letter-based regression dynamic

# Create a list of unique first letters of variables
X = df.drop(['UID','Rating','target'], axis=1)
y = df["target"]

var_letters = []

for i in X.columns:
    var_letters.append(i[0:1])  # Append first character

var_letters = list(np.unique(var_letters))  # List of unique variable letters



### Variable Group Regressions & PCA Code

In [4]:
################################################################################
### Hidden: Code for the regressions on variable groups A, B, C, D
################################################################################


# ### Linear Regression with A Variables

# # Unique IDs
# ids = df.UID.unique()

# # Run linear regressions for each UID
# op = pd.DataFrame
# intercept = []
# coefficients=[]
# UID = []
# for i in ids:
#     df_i = df[df.UID == i]              # Create dataframe for current user id
#     X = df_i.drop(['UID','B1', 'B2', 'B3', 'B4','C1', 'C2', 'C3', 'C4','D1', 'D2', 'D3', 'D4','Rating','target'], axis=1)  # df input variables only
#     y = df_i['target']                  # Series of target variable
#     reg = LinearRegression().fit(X, y)  # Fit linear regression
#     reg.score(X, y)                     # Score regression model
#     unique_id=df_i['UID'].unique()      # Saves current user id
#     const = reg.intercept_              # Save intercept of the regression model
#     coef = reg.coef_                    # Coefficients of regression model
#     UID.append(unique_id)               # Append current user id
#     intercept.append(const)             # Append current intercept
#     coefficients.append(coef)           # Append current regression coefficients

# # Convert newly created lists into dataframes
# intercep_new = pd.DataFrame(intercept)
# coefficients_new = pd.DataFrame(coefficients)
# UID_new = pd.DataFrame(UID)

# # Get columns names
# colNames = df.drop(['B1', 'B2', 'B3', 'B4','C1', 'C2', 'C3', 'C4','D1', 'D2', 'D3', 'D4','Rating', 'target'], axis=1).columns
# colNames = colNames.insert(1, 'Const')
# colNames

# # Concatenate the new dataframes and add column names
# op = pd.concat([UID_new,intercep_new, coefficients_new], axis=1)
# op.columns = colNames
# op_A = op.iloc[:,2:6]


# #******************************************************************************#
# ### Linear Regression with B Variables

# # Unique IDs
# ids = df.UID.unique()

# # Run linear regressions for each UID
# op = pd.DataFrame
# intercept = []
# coefficients=[]
# UID = []
# for i in ids:
#     df_i = df[df.UID == i]              # Create dataframe for current user id
#     X = df_i.drop(['UID','A1', 'A2', 'A3', 'A4','C1', 'C2', 'C3', 'C4','D1', 'D2', 'D3', 'D4','Rating','target'], axis=1)  # df input variables only
#     y = df_i['target']                  # Series of target variable
#     reg = LinearRegression().fit(X, y)  # Fit linear regression
#     reg.score(X, y)                     # Score regression model
#     unique_id=df_i['UID'].unique()      # Saves current user id
#     const = reg.intercept_              # Save intercept of the regression model
#     coef = reg.coef_                    # Coefficients of regression model
#     UID.append(unique_id)               # Append current user id
#     intercept.append(const)             # Append current intercept
#     coefficients.append(coef)           # Append current regression coefficients

# # Convert newly created lists into dataframes
# intercep_new = pd.DataFrame(intercept)
# coefficients_new = pd.DataFrame(coefficients)
# UID_new = pd.DataFrame(UID)

# # Get columns names
# colNames = df.drop(['A1', 'A2', 'A3', 'A4','C1', 'C2', 'C3', 'C4','D1', 'D2', 'D3', 'D4','Rating', 'target'], axis=1).columns
# colNames = colNames.insert(1, 'Const')
# colNames

# # Concatenate the new dataframes and add column names
# op = pd.concat([UID_new,intercep_new, coefficients_new], axis=1)
# op.columns = colNames
# op_B = op.iloc[:,2:6]


# #******************************************************************************#
# ### Linear Regression with C Variables

# # Unique IDs
# ids = df.UID.unique()

# # Run linear regressions for each UID
# op = pd.DataFrame
# intercept = []
# coefficients=[]
# UID = []
# for i in ids:
#     df_i = df[df.UID == i]              # Create dataframe for current user id
#     X = df_i.drop(['UID', 'A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'D1', 'D2', 'D3', 'D4', 'Rating', 'target'], axis=1)  # df input variables only
#     y = df_i['target']                  # Series of target variable
#     reg = LinearRegression().fit(X, y)  # Fit linear regression
#     reg.score(X, y)                     # Score regression model
#     unique_id=df_i['UID'].unique()      # Saves current user id
#     const = reg.intercept_              # Save intercept of the regression model
#     coef = reg.coef_                    # Coefficients of regression model
#     UID.append(unique_id)               # Append current user id
#     intercept.append(const)             # Append current intercept
#     coefficients.append(coef)           # Append current regression coefficients

# # Convert newly created lists into dataframes
# intercep_new = pd.DataFrame(intercept)
# coefficients_new = pd.DataFrame(coefficients)
# UID_new = pd.DataFrame(UID)

# # Get columns names
# colNames = df.drop(['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'D1', 'D2', 'D3', 'D4', 'Rating', 'target'], axis=1).columns
# colNames = colNames.insert(1, 'Const')
# colNames

# # Concatenate the new dataframes and add column names
# op = pd.concat([UID_new,intercep_new, coefficients_new], axis=1)
# op.columns = colNames
# op_C = op.iloc[:,2:6]



# #******************************************************************************#
# ### Linear Regression with D Variables

# # Unique IDs
# ids = df.UID.unique()

# # Run linear regressions for each UID
# op = pd.DataFrame
# intercept = []
# coefficients=[]
# UID = []
# for i in ids:
#     df_i = df[df.UID == i]              # Create dataframe for current user id
#     X = df_i.drop(['UID', 'A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'Rating', 'target'], axis=1)  # df input variables only
#     y = df_i['target']                  # Series of target variable
#     reg = LinearRegression().fit(X, y)  # Fit linear regression
#     reg.score(X, y)                     # Score regression model
#     unique_id=df_i['UID'].unique()      # Saves current user id
#     const = reg.intercept_              # Save intercept of the regression model
#     coef = reg.coef_                    # Coefficients of regression model
#     UID.append(unique_id)               # Append current user id
#     intercept.append(const)             # Append current intercept
#     coefficients.append(coef)           # Append current regression coefficients

# # Convert newly created lists into dataframes
# intercep_new = pd.DataFrame(intercept)
# coefficients_new = pd.DataFrame(coefficients)
# UID_new = pd.DataFrame(UID)

# # Get columns names
# colNames = df.drop(['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'Rating', 'target'], axis=1).columns
# colNames = colNames.insert(1, 'Const')
# colNames

# # Concatenate the new dataframes and add column names
# op = pd.concat([UID_new,intercep_new, coefficients_new], axis=1)
# op.columns = colNames
# op_D = op.iloc[:,2:6]


# #******************************************************************************#
# ### Replace 1's w/ Regression Coefficients in Original Data

# # Concatenate regression dataframes
# all_cfs = pd.concat([op_A, op_B, op_C, op_D], axis=1)

# # Replace 1's w/ regression coefficients by column
# cfs_cols = all_cfs.columns

# for col in cfs_cols:
#     for i in range(1,len(all_cfs)+1):
#         df.loc[df['UID'] == i,[col]] = df.loc[df['UID'] == i,[col]].replace(1,all_cfs.loc[i-1,col])

# df.head()



################################################################################
## PCA on Regression Coefficients
################################################################################


# #******************************************************************************#
# # Comparing covariance and correlation PCA.  We should do correlation PCA.

# # Create PCA dataframe
# df_fct = op.drop(['UID','Const'], axis=1)

# # Standardize df_fct
# # df_pca = stats.zscore(df_fct)  # Didn't need to worry about standardizing.
# # Could build in a correlation/covariance PCA thing using ranges of the variables

# # Create PCA object
# pca = PCA(random_state=123)

# # Get principal components
# pca.fit(df_fct)

# # Get scores
# pca.transform(df_fct)

# # Components needed
# pcs_needed = len(np.where(pca.explained_variance_ >= 1)[0])

# # Save scores for PCs w/ eignvalues >=1 as dataframe for clustering
# scores = pd.DataFrame(pca.transform(df_fct))
# scores = scores.iloc[:,0:pcs_needed]

# # How many observations
# n_samples = pca.components_.shape[0]

# # Transpose the principal components
# murph = pca.components_.T
# # Center the data
# murph -= np.mean(murph, axis=0)
# # Compute the covariance matrix
# cov_matrix = np.dot(murph.T, murph) / n_samples

# for eigenvector in pca.components_:
#     print(np.dot(eigenvector.T, np.dot(cov_matrix, eigenvector)))

# n_samples = df_fct.shape[0]

# pca = PCA(random_state=123)
# X_transformed = pca.fit_transform(df_fct)

# # This is the explained variance.  They are big numbers because this is Cov. PCA
# X_centered = df_fct - np.mean(df_fct, axis=0)
# cov_matrix = np.dot(X_centered.T, X_centered) / n_samples
# eigenvalues = pca.explained_variance_
# for eigenvalue, eigenvector in zip(eigenvalues, pca.components_):
#     print(eigenvalue)

# # This is the same thing as the code above but built into sklearn
# pca.explained_variance_
# # These are the eigenvalues of the covariance matrix
# # They don't have values near 1-ish, so I need to understand why.


# #******************************************************************************#
# # Biplots for comparing correlation and covariance PCA for this data

# # !pip install pca
# from pca import pca

# # Correlation PCA & biplot
# model = pca(n_components=4)
# results = model.fit_transform(df_pca)
# fig, ax = model.biplot(n_feat=16, cmap=None, label=False, legend=False)

# # Covariance PCA & biplot
# model2 = pca(n_components=4)
# results2 = model2.fit_transform(df_fct)
# fig, ax = model2.biplot(n_feat=16, cmap=None, label=False, legend=False)

# # Create PCA dataframe
# df_fct = op.drop(['UID','Const'], axis=1)

# # Standardize df_fct - Need to standardize to use eigenvalues >= 1
# df_fct = stats.zscore(df_fct)

# # Create PCA object
# pca = PCA(random_state=123)

# # Get principal components & scores
# pca.fit_transform(df_fct)

# # Components needed
# pcs_needed = len(np.where(pca.explained_variance_ >= 1)[0])

# # Save scores for PCs w/ eignvalues >=1 as dataframe for clustering
# scores = pd.DataFrame(pca.transform(df_fct))
# scores = scores.iloc[:,0:pcs_needed]
# expl_var = round(pca.explained_variance_ratio_.cumsum()[pcs_needed-1]*100, 2)

# print("Principal Components Used: ", pcs_needed, sep='')
# print("Variance Explained: ", expl_var, "%", sep='')

### Experimenting with SMOTE for synthetic data creation

In [117]:
################################################################################
# EXPERIMENTING WITH SMOTE
################################################################################

# Maximum number of clusters
max_clusters = 6

for n in range(2, max_clusters+1):
    
    sw = []
    
    # Create clustering objects
    cls1 = KMeans(n_clusters=n, random_state=0)
    cls2 = KMedoids(n_clusters=n, random_state=0)
    cls3 = AgglomerativeClustering(n_clusters=n,
                                   affinity='euclidean',
                                   linkage='ward')
        # Agglomerative clustering: if linkage=ward, affinity must be Euclidean
    cls_algs = [['kMeans', cls1],
                ['kMedoids', cls2],
                ['Hierarchical', cls3]]
    
    # Fit and score clustering solutions for i clusters w/ each algorithm
    for cls in cls_algs:
        
        # Fit the model to the factor analysis scores
        cls[1].fit(scores)
        
        # List of assigned clusters
        clusters = cls[1].fit_predict(scores)
        
        # Silhouette scores for each solution
        silhouette_avg = silhouette_score(scores,clusters)
        
        # Store solution info
        algorithm = cls[0]
        n_stats = [algorithm, n, silhouette_avg, clusters]
        sw.append(n_stats)

    # Reorder cluster lists by descending silhouette scores.
    # Clusters in first element should be assigned to training data.
    sw = sorted(sw, key=itemgetter(2), reverse=True)
    op[f'Optimal {sw[0][1]} cluster solution ({sw[0][0]})'] = sw[0][3] + 1

import imblearn
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

X = op.filter(regex='^[a-zA-Z][0-9]')
y = op.iloc[:,18]

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

op.head()

X['Optimal 2 cluster solution (Hierarchical)'] = y
X['UID'] = op.UID
X['Const'] = op.Const
# X = X.iloc[:, np.r_[-2,-1,0:-2]]

cols = X.columns.tolist()
cols = cols[-2:] + cols[:-2]

X = X[cols]

op = X

op.shape

op.head()

for i in range(18,19):


    df_cl = op.iloc[:,np.r_[2:18,i]]  # i is the current cluster solution

    #**********************************************************************#

    # Split data into 70% training, 30% validation
    train, valid = train_test_split(df_cl, test_size=0.30, random_state=123)

    # X is unlabeled training data, y is true training labels 
    X, y = train.iloc[:,0:-1], train.iloc[:,-1]

    X_valid, y_valid = valid.iloc[:,0:-1], valid.iloc[:,-1]

    #**********************************************************************#

    # Get variable importances

    clf1 = RandomForestClassifier(random_state=0)
    clf2 = GradientBoostingClassifier(random_state=0)

    classifiers = [['rf', clf1], ['gbt', clf2]]

    for classifier in classifiers:    
        # Fit classifier to training data
        classifier[1].fit(X,y)    

    # Create variable importance dataframe
    num_vars = list(range(1,len(clf1.feature_importances_)+1))
    importance = pd.DataFrame({'variable': num_vars,
                               'rf': clf1.feature_importances_,
                               'gbt': clf2.feature_importances_,})

    # Average variable importance of rf and gbt models
    importance['avg'] = (importance['rf']+importance['gbt'])/2

    # Put avg importances on a scale from 0 to 1 to make it easier to visualize
    importance['Relative Importance'] = np.interp(importance['avg'],
                                                  (importance['avg'].min(),
                                                   importance['avg'].max()),
                                                  (0, 1))

    # View top 10 variables when RF and GBT models are averaged
    top_10_avg = importance.sort_values(by='avg', ascending=False)[['avg','Relative Importance']].head(10)

    # Add variable rank column to dataframe
    importance_rank = num_vars
    importance = importance.sort_values(by='Relative Importance', ascending=False)
    importance['rank'] = importance_rank
    importance.reset_index(inplace=True)

    # Save index of top 5 variables (not the variable number!)
    top_5 = importance[importance['rank'] <= 5]['index']

    #**********************************************************************#
    # Average and Standard Deviations for each cluster/variable combination
    # For cluster 1 of 2, calculate the average and stdev for each variable
    # For cluster 2 of 2, calculate the average and stdev for each variable
    # Etc.

#     if n == max_clusters:

#         cls_avg_list = []

#         # Take the mean of every variable for each cluster
#         for k in range(1, df_cl_cons.iloc[:,-1].max()+1):
#             cls_mean = df_cl_cons[df_cl_cons.iloc[:,-1] == k].iloc[:,0:-1].mean()
#             cls_mean = cls_mean.append(pd.Series({"Count":df_cl[df_cl.iloc[:,-1] == k].iloc[:,0:-1].shape[0]}))
#             cls_avg_list.append(cls_mean)
#             cls_std = df_cl_cons[df_cl_cons.iloc[:,-1] == k].iloc[:,0:-1].std()
#             cls_avg_list.append(cls_std)
#             # NaN means there is either only 1 observation in that cluster or none.
        
#         # Convert to dataframe and transpose
#         cls_averages = pd.DataFrame(cls_avg_list)
#         cls_averages = cls_averages.T

#         # Create helpful column names (Cluster # of total_#)
#         col_names = []
        
#         for col in range(1, k+1):
#             new_name1 = f"Avg cluster {col}/{k}"
#             col_names.append(new_name1)
#             new_name2 = f"Std cluster {col}/{k}"
#             col_names.append(new_name2)            
            
#         # Rename columns
#         cls_averages.columns = col_names

#         cls_averages_all = pd.concat([cls_averages_all, cls_averages], axis=1)
        
        


    #**********************************************************************#
    # Convert data to binary, train classifiers, score validation, create maps

    # Convert X, X_valid, and df_cl predictors to all 1 and -1
#     X = (X.mask(df > 0, other=1, inplace=False)
#          .mask(df <= 0, other=-1, inplace=False))
#     X_valid = (X_valid.mask(df > 0, other=1, inplace=False)
#                .mask(df <= 0, other=-1, inplace=False))
    all_data_masked = (df_cl.iloc[:,0:-1]) #.mask(df > 0, other=1, inplace=False)
#                        .mask(df <= 0, other=-1, inplace=False))

    map_collection = []

    # Retrain on the 2-5 most important variables
    for j in range(2,6):

        clf_scores = []

        clf1 = RandomForestClassifier(random_state=0)
        clf2 = GradientBoostingClassifier(random_state=0)
        clf3 = SVC(random_state=0)
        clf4 = KNeighborsClassifier()

        classifiers = [['rf', clf1], ['gbt', clf2], ['svc', clf3], ['knn', clf4]]

        # Fit each classifier to the current variable/cluster combination
        for classifier in classifiers:

            # Fit classifier to training data
            classifier[1].fit(X.iloc[:,np.r_[top_5[0:j]]],y)
            print(classifier[1].predict(X.iloc[:,np.r_[top_5[0:j]]]))

            # Store classifier-specific results [algorithm object, classifier name, scores]
            results = [classifier[1],
                       classifier[0],
                       classifier[1].score(X_valid.iloc[:,np.r_[top_5[0:j]]],y_valid)]

            # Overall classifier results
            clf_scores.append(results)

        # Sort classifier accuracy in descending order
        clf_scores = sorted(clf_scores, key=itemgetter(2), reverse=True)
        # clf_scores[0][0] is the best model

        # Fit the best model on all data
        best_model = clf_scores[0][0].fit(all_data_masked.iloc[:,np.r_[top_5[0:j]]], df_cl.iloc[:,-1])
        
        print(j)
        print(clf_scores)

#         #******************************************************************#
#         # Create mappings

#         # Creates grid of dimension j
#         grid = pd.DataFrame(list(itertools.product([-1,1], repeat=j)))

#         grid.columns = all_data_masked.iloc[:,np.r_[top_5[0:j]]].columns

#         # This is the best model predicting the grid
#         preds = best_model.predict(grid)            

#         # Add to grid dataframe
#         grid['Predicted Cluster'] = preds

#         # Change grid to mapping to fit into the rest of the code
#         mapping = grid

#         # Save current mapping to map collection for this cluster solution
#         map_collection.append(mapping)

#         # Write each dataframe to a different worksheet.
#         mapping.to_excel(writer, index=False, sheet_name=f"{df_cl.columns[-1][8:17]}s, {j} vars, {round(clf_scores[0][2]*100)}% Acc.")

#     all_maps.append(map_collection)

# op.to_excel(writer, index=False, sheet_name="All Regressions, Clusters")


# # Add averages for all observations to cls_averages_all before exporting
# all_obs = []

# # Variable means for all observations
# all_obs_mean = list(op.filter(regex='^[a-zA-Z][0-9]').mean().values)
# all_obs_mean.insert(0,op['Const'].mean())
# all_obs.append(all_obs_mean)

# cls_averages_all['new_col'] = pd.Series(list(op.filter(regex='^[a-zA-Z][0-9]').mean()).values

# # Variable standard deviations for all observations
# all_obs_std = list(op.filter(regex='^[a-zA-Z][0-9]').std().values)
# all_obs_std.insert(0,op['Const'].std())
# all_obs.append(all_obs_std)


# # Save as dataframe and append to all cls_averages_all dataframe
# all_obs_cols = list(op.filter(regex='^[a-zA-Z][0-9]').columns)
# all_obs_cols.insert(0, "Const")
# all_obs_df = pd.DataFrame(all_obs, columns=all_obs_cols)
# all_obs_df = all_obs_df.T
# all_obs_cols = ['All obs avg', 'All obs stdev']
# all_obs_df.columns = all_obs_cols
# cls_averages_all = pd.concat([cls_averages_all, all_obs_df], axis=1)


# cls_averages_all.to_excel(writer, sheet_name="Cluster Avgs and StDevs")

# writer.save()

<b>Scenario Analysis</b>

In [46]:
df.head()

,UID,A1,A2,A3,A4,B1,B2,B3,B4,C1,C2,C3,C4,D1,D2,D3,D4,Rating,target
0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,100,100.000507
1,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,100,100.000668
2,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0.000386
3,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0.000067
4,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0.000382


In [19]:
# np.where(condition, value if condition is true, value if condition is false)

# create a list of the column categories

cat_A = ['A1', 'A2', 'A3', 'A4']
cat_B = ['B1', 'B2', 'B3', 'B4']
cat_C = ['C1', 'C2', 'C3', 'C4']
cat_D = ['D1', 'D2', 'D3', 'D4']

# create a list of our conditions
cat_A_conditions = [
    (df['A1'] == 1),
    (df['A2'] == 1),
    (df['A3'] == 1),
    (df['A4'] == 1),
    (df['A1']==0) & (df['A2']==0) & (df['A3']==0) & (df['A4']==0),
    ]

cat_B_conditions = [    
    (df['B1'] == 1),
    (df['B2'] == 1),
    (df['B3'] == 1),
    (df['B4'] == 1),
    (df['B1']==0) & (df['B2']==0) & (df['B3']==0) & (df['B4']==0),
    ]    

cat_C_conditions = [    
    (df['C1'] == 1),
    (df['C2'] == 1),
    (df['C3'] == 1),
    (df['C4'] == 1),
    (df['C1']==0) & (df['C2']==0) & (df['C3']==0) & (df['C4']==0),
    ]

cat_D_conditions = [    
    (df['D1'] == 1),
    (df['D2'] == 1),
    (df['D3'] == 1),
    (df['D4'] == 1),  
    (df['D1']==0) & (df['D2']==0) & (df['D3']==0) & (df['D4']==0),
    ]

# create a list of the values we want to assign for each condition
cat_A_values = ['A1', 'A2', 'A3', 'A4', 'A0']
cat_B_values = ['B1', 'B2', 'B3', 'B4','B0']
cat_C_values = ['C1', 'C2', 'C3', 'C4','C0']
cat_D_values = ['D1', 'D2', 'D3', 'D4','D0']
    
df['cat_A_scenario'] = np.select(cat_A_conditions, cat_A_values)
df['cat_B_scenario'] = np.select(cat_B_conditions, cat_B_values)
df['cat_C_scenario'] = np.select(cat_C_conditions, cat_C_values)
df['cat_D_scenario'] = np.select(cat_D_conditions, cat_D_values)

<b> Example of Scenario where Variable A1 has a 1 </b>

In [20]:
cat_A = df['cat_A_scenario'].unique()
# list of independant variables for regression
fields = df.columns[1:17]
dep_var = df['noise']

In [21]:
df_A1 = df[df['cat_A_scenario']=='A1']
df_A1

,UID,A1,A2,A3,A4,B1,B2,B3,B4,C1,...,D1,D2,D3,D4,Rating,noise,cat_A_scenario,cat_B_scenario,cat_C_scenario,cat_D_scenario
0,1,1,0,0,0,0,0,0,1,1,...,0,0,0,0,100,100.000893,A1,B4,C1,D0
1,1,1,0,0,0,0,0,1,0,0,...,1,0,0,0,100,100.000783,A1,B3,C0,D1
13,1,1,0,0,0,0,0,0,0,1,...,0,0,0,1,100,100.000825,A1,B0,C1,D4
16,1,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0.000479,A1,B4,C4,D3
17,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,100,100.000252,A1,B1,C3,D4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383,100,1,0,0,0,0,0,0,1,0,...,0,0,0,1,5,5.000552,A1,B4,C0,D4
2385,100,1,0,0,0,0,0,1,0,0,...,0,0,1,0,5,5.000693,A1,B3,C2,D3
2393,100,1,0,0,0,0,0,0,1,0,...,0,1,0,0,5,5.000722,A1,B4,C4,D2
2398,100,1,0,0,0,1,0,0,0,0,...,0,0,0,0,5,5.000264,A1,B1,C4,D0


In [24]:
X =df_A1[['B1', 'B2', 'B3', 'B4','C1', 'C2', 'C3', 'C4','D1', 'D2', 'D3', 'D4']]
y= df_A1['noise']
reg = LinearRegression().fit(X, y)
reg.score(X, y)
const = reg.intercept_
coef = reg.coef_
print(const)
print(coef)

33.91035518762444
[ -3.84235504 -13.45891789  -5.98115784  -1.00151876  -5.2084735
  -0.95495595  -3.15645822  -7.2888877   -2.32297686  -3.7339574
  -5.08132817   1.23153583]


In [23]:
# scratch pad, not finished
for i in cat_A:
    df_i = df[df.cat_A_scenario == i]
    X =df_i[df_i.columns[1:17]]
    y= df_i['noise']
    reg = LinearRegression().fit(X, y)
    reg.score(X, y)
    const = reg.intercept_
    coef = reg.coef_
    intercept.append(const)
    coefficients.append(coef)
#print(UID)    
intercep_new = pd.DataFrame(intercept)
coefficients_new = pd.DataFrame(coefficients)